<a href="https://colab.research.google.com/github/RoyTng/ADALL-Project/blob/main/My_ADALL_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Predictive Maintenance for Servers Downtime in Data Centre.

### Business Problem
Predict near-term server failure risk to prevent SLA breaches and revenue loss from unplanned downtime.

### Config Cell
github_raw_url,
random_state=42,
target_col="Machine failure",
and drop_cols=["TWF","HDF","PWF","OSF","RNF"]

In [1]:
# Core libraries
import pandas as pd
import numpy as np
import seaborn as sns
# Visualisation
import matplotlib.pyplot as plt
# Modelling and preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

In [2]:
# Example: Replace this with the raw URL of your GitHub file
github_raw_url = 'https://raw.githubusercontent.com/RoyTng/ADALL-Project/refs/heads/main/Ai4i.csv'
try:
    df = pd.read_csv(github_raw_url)
    print("Successfully loaded data from GitHub!")
    display(df.head())
except Exception as e:
    print(f"Error loading data: {e}")
    print("Please ensure the URL is correct and the file format is compatible with `pd.read_csv`.")


Successfully loaded data from GitHub!


,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   UDI                      10000 non-null  int64  
 1   Product ID               10000 non-null  object 
 2   Type                     10000 non-null  object 
 3   Air temperature [K]      10000 non-null  float64
 4   Process temperature [K]  10000 non-null  float64
 5   Rotational speed [rpm]   10000 non-null  int64  
 6   Torque [Nm]              10000 non-null  float64
 7   Tool wear [min]          10000 non-null  int64  
 8   Machine failure          10000 non-null  int64  
 9   TWF                      10000 non-null  int64  
 10  HDF                      10000 non-null  int64  
 11  PWF                      10000 non-null  int64  
 12  OSF                      10000 non-null  int64  
 13  RNF                      10000 non-null  int64  
dtypes: float64(3), int64(9)

In [4]:
from google.colab import userdata
from openai import OpenAI

# Load key from Google Colab Secrets
api_key = userdata.get('OPENAI_API_KEY')

client = OpenAI(
    api_key=api_key,
)

In [6]:
#generate a preview of ten rows as text first, so that we can use it for sending to LLM API later.
data_preview = df.head(50).to_string()
print(data_preview)

    UDI Product ID Type  Air temperature [K]  Process temperature [K]  Rotational speed [rpm]  Torque [Nm]  Tool wear [min]  Machine failure  TWF  HDF  PWF  OSF  RNF
0     1     M14860    M                298.1                    308.6                    1551         42.8                0                0    0    0    0    0    0
1     2     L47181    L                298.2                    308.7                    1408         46.3                3                0    0    0    0    0    0
2     3     L47182    L                298.1                    308.5                    1498         49.4                5                0    0    0    0    0    0
3     4     L47183    L                298.2                    308.6                    1433         39.5                7                0    0    0    0    0    0
4     5     L47184    L                298.2                    308.7                    1408         40.0                9                0    0    0    0    0    0
5   

### Data Profiling

In [7]:
# import pandas as pd
# import numpy as np
from io import StringIO

# ---------------------------
# Generate a full dataset profile
# ---------------------------

buffer = StringIO()

# dtypes
buffer.write("=== DTYPES ===\n")
buffer.write(df.dtypes.to_string())
buffer.write("\n\n")

# numeric describe
buffer.write("=== NUMERIC DESCRIBE ===\n")
buffer.write(df.describe().to_string())
buffer.write("\n\n")

# categorical describe
buffer.write("=== CATEGORICAL DESCRIBE ===\n")
try:
    buffer.write(df.describe(include='object').to_string())
except:
    buffer.write("No categorical columns")
buffer.write("\n\n")

# null summary
buffer.write("=== NULL SUMMARY ===\n")
null_summary = (
    df.isna().sum().to_frame("null_count")
    .assign(null_pct=lambda x: x["null_count"]/len(df))
)
buffer.write(null_summary.to_string())
buffer.write("\n\n")

# unique cardinality
buffer.write("=== UNIQUE VALUES PER COLUMN ===\n")
buffer.write(df.nunique().to_frame("unique_count").to_string())
buffer.write("\n\n")

# correlation matrix
buffer.write("=== CORRELATIONS (NUMERIC ONLY) ===\n")
buffer.write(df.corr(numeric_only=True).round(3).to_string())
buffer.write("\n\n")

# value counts for categoricals
buffer.write("=== VALUE COUNTS (TOP 20 PER CATEGORICAL COLUMN) ===\n")
cat_cols = df.select_dtypes(include='object').columns
if len(cat_cols) > 0:
    for col in cat_cols:
        buffer.write(f"\nColumn: {col}\n")
        vc = df[col].value_counts().head(20)
        buffer.write(vc.to_string())
        buffer.write("\n")
else:
    buffer.write("No categorical columns\n")
buffer.write("\n")

# --------- FIXED OUTLIER COMPUTATION (NO BOOLEANS) ---------
buffer.write("=== OUTLIER SUMMARY (IQR METHOD) ===\n")
num_cols = df.select_dtypes(include=['number']).columns  # exclude booleans
Q1 = df[num_cols].quantile(0.25)
Q3 = df[num_cols].quantile(0.75)
IQR = Q3 - Q1
outliers = ((df[num_cols] < (Q1 - 1.5*IQR)) | (df[num_cols] > (Q3 + 1.5*IQR))).sum()
buffer.write(outliers.to_string())
buffer.write("\n\n")

# leakage scan: columns with all unique values
buffer.write("=== POSSIBLE LEAKAGE COLUMNS (UNIQUE FOR EACH ROW) ===\n")
leak_cols = df.columns[df.nunique() == len(df)]
buffer.write(str(list(leak_cols)))
buffer.write("\n\n")

# shape, duplicates, constant cols
buffer.write("=== SHAPE / DUPLICATES / CONSTANT COLUMNS ===\n")
dup_count = df.duplicated().sum()
constant_cols = df.columns[df.nunique() == 1].tolist()
buffer.write(f"Rows: {len(df)}, Columns: {df.shape[1]}\n")
buffer.write(f"Duplicate rows: {dup_count}\n")
buffer.write(f"Constant columns: {constant_cols}\n\n")

# Final text
payload_text = buffer.getvalue()

print(payload_text)


=== DTYPES ===
UDI                          int64
Product ID                  object
Type                        object
Air temperature [K]        float64
Process temperature [K]    float64
Rotational speed [rpm]       int64
Torque [Nm]                float64
Tool wear [min]              int64
Machine failure              int64
TWF                          int64
HDF                          int64
PWF                          int64
OSF                          int64
RNF                          int64

=== NUMERIC DESCRIBE ===
               UDI  Air temperature [K]  Process temperature [K]  Rotational speed [rpm]   Torque [Nm]  Tool wear [min]  Machine failure           TWF           HDF           PWF           OSF          RNF
count  10000.00000         10000.000000             10000.000000            10000.000000  10000.000000     10000.000000     10000.000000  10000.000000  10000.000000  10000.000000  10000.000000  10000.00000
mean    5000.50000           300.004930               310

### LLM-assisted Data Profiling

In [8]:
response = client.responses.create(
    model="gpt-5-mini",
    instructions="""
You are an expert data scientist with extensive knowledge of tree-based models.
Always justify recommendations using reasoning trace based ONLY on the dataset profile.
""",
    input=f"""
Dataset info: {payload_text}\n
Questions:\n
1. Based on the dataset profile, what data quality issues should be resolved before modelling?
Provide a priority list and justify each item. \n
2. Which columns appear redundant, correlated, or likely to cause leakage?
Explain why each is problematic. \n
Next: Provide a python script to handle the identified issues.
Define one helper function for each issue.
Then define a wrapper function that calls these helper with true false option as user choice
Provide a single line of code to run the overall wrapper function.
Do not encode categorical columns or model first.
""")

print(response.output_text)


1) Data-quality issues to resolve before modeling — priority list (highest → lowest), with justification from the dataset profile

- Remove uniquely identifying / per-row ID fields (highest priority)
  - Why: UDI and Product ID are unique for every row (unique_count = 10000 for both; "POSSIBLE LEAKAGE COLUMNS" lists them). Identifiers are not predictive of failure in a generalizable way and will cause models to memorize rows (leakage).
  - Profile trace: unique_count UDI = 10000, Product ID = 10000; flagged as possible leakage.

- Remove or exclude explicit failure-type flags that leak the target
  - Why: The binary columns TWF, HDF, PWF, OSF, RNF are strongly associated with the target Machine failure (correlations with Machine failure: TWF 0.363, HDF 0.576, PWF 0.523, OSF 0.531) and likely represent the same event or post-failure labels. Including them would leak the answer to the model.
  - Profile trace: high correlations TWF/HDF/PWF/OSF with Machine failure shown in "CORRELATIONS"

### 1) Data-quality issues to resolve (priority order)
Target leakage via failure-mode labels (TWF/HDF/PWF/OSF/RNF): these columns are explicit failure indicators recorded alongside the target, so using them as predictors would let a model “cheat” by reading near-direct labels rather than learning from sensor signals.

Identifier columns (UDI, Product ID): these are IDs, not physical condition signals; they can let tree models memorize per-unit patterns (especially if a given ID is associated with failures) and inflate validation performance without improving real generalization.
​

Label consistency between Machine failure and failure modes: because the dataset includes both the overall failure label (Machine failure) and the specific mode flags, you should validate that Machine failure matches the logical OR / max of the mode columns; inconsistencies are a data-quality defect that can confuse training and evaluation.
​

Duplicate rows / duplicate IDs: since UDI appears to be a row identifier, duplicates (by full row or by ID) can cause leakage across splits (the same record appearing in train and test) and should be checked and removed or deduplicated.
​

Missing / invalid values and data types: the file is numeric-heavy with units in column names (e.g., temperatures in K, speed in rpm), so you should enforce numeric dtypes and check for missing/blank/non-numeric entries introduced during CSV handling (e.g., BOM characters in header).
​

Outliers / physically implausible values: tree-based models can be sensitive to extreme values because they create very “pure” splits; check for negative values in fields that should be non-negative (speed/torque/tool wear) and very extreme tails that may represent sensor glitches.
​

Class imbalance sanity check: the target is binary with relatively few “1”s visible in the sample (most rows show Machine failure=0), so you should quantify imbalance before splitting to avoid splits with too few positives.

### 2) Redundant, correlated, or leakage-prone columns
TWF, HDF, PWF, OSF, RNF: Leakage + redundancy because they are specific failure labels that are logically tied to the overall Machine failure target.

UDI, Product ID: High leakage/generalization risk because they are identifiers rather than physical signals, and tree models can overfit via ID-specific rules.
​

Air temperature [K] and Process temperature [K]: Likely correlated because both are temperature measures recorded at the same time; high correlation can make interpretation less stable (importance split across features) even if it’s not “wrong.”
​

Rotational speed [rpm] and Torque [Nm]: Often correlated/linked by operating regime (load and control response), so you should at least quantify correlation to understand redundancy and interpretation.
​

Type vs Product ID: Potential redundancy because product identifiers can implicitly encode type/tier; dropping IDs while keeping Type preserves generalizable signal